# Core Utilities

> Core utilities and data models for nbdev project overview generation

In [ ]:
#| default_exp core

In [ ]:
#| export
from __future__ import annotations
from pathlib import Path
from typing import Union, Dict, Any, List, Optional, Tuple
from dataclasses import dataclass, field
from nbdev.config import get_config
import json
from fastcore.basics import *
from fastcore.imports import *

## Data Models

First, let's define the data models we'll use throughout the project:

In [ ]:
#| export
@dataclass
class NotebookInfo:
    "Information about a single notebook"
    path: Path                              # Path to the notebook file
    name: str                              # Notebook filename without extension
    title: Optional[str] = None            # H1 title from first cell
    description: Optional[str] = None      # Blockquote description from first cell
    export_module: Optional[str] = None    # Module name from default_exp
    
    @property
    def relative_path(self) -> Path:       # Path relative to nbs directory
        "Get path relative to nbs directory"
        cfg = get_config()
        return self.path.relative_to(cfg.nbs_path)

In [ ]:
#| export
@dataclass
class DirectoryInfo:
    "Information about a directory in the nbs folder"
    path: Path                                      # Path to the directory
    name: str                                       # Directory name
    notebook_count: int = 0                         # Number of notebooks in directory
    description: Optional[str] = None               # Description from folder's main notebook
    subdirs: List[DirectoryInfo] = field(default_factory=list)  # Subdirectories
    notebooks: List[NotebookInfo] = field(default_factory=list) # Notebooks in this directory
    
    @property
    def total_notebook_count(self) -> int:          # Total notebooks including subdirs
        "Get total notebook count including subdirectories"
        return self.notebook_count + sum(d.total_notebook_count for d in self.subdirs)

## Path Utilities

In [ ]:
#| export
def get_notebook_files(path: Path = None,           # Directory to search (defaults to nbs_path)
                      recursive: bool = True        # Search subdirectories
                      ) -> List[Path]:              # List of notebook paths
    "Get all notebook files in a directory"
    if path is None:
        cfg = get_config()
        path = cfg.nbs_path
    
    pattern = '**/*.ipynb' if recursive else '*.ipynb'
    notebooks = []
    
    for nb_path in path.glob(pattern):
        # Skip checkpoint notebooks and hidden files
        if '.ipynb_checkpoints' not in str(nb_path) and not nb_path.name.startswith('.'):
            notebooks.append(nb_path)
    
    return sorted(notebooks)

In [ ]:
#| export
def get_subdirectories(path: Path = None,           # Directory to search (defaults to nbs_path)
                      recursive: bool = False       # Include all nested subdirectories
                      ) -> List[Path]:              # List of directory paths
    "Get subdirectories in a directory"
    if path is None:
        cfg = get_config()
        path = cfg.nbs_path
    
    subdirs = []
    
    if recursive:
        for item in path.rglob('*'):
            if item.is_dir() and '.ipynb_checkpoints' not in str(item) and not item.name.startswith('.'):
                subdirs.append(item)
    else:
        for item in path.iterdir():
            if item.is_dir() and '.ipynb_checkpoints' not in str(item) and not item.name.startswith('.'):
                subdirs.append(item)
    
    return sorted(subdirs)

## Notebook Reading Utilities

In [ ]:
#| export
def read_notebook(path: Path                    # Path to notebook file
                 ) -> Dict[str, Any]:           # Notebook content as dict
    "Read a notebook file and return its content"
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

In [ ]:
#| export
def get_cell_source(cell: Dict[str, Any]        # Notebook cell
                   ) -> str:                    # Cell source as string
    "Get source from a notebook cell"
    source = cell.get('source', [])
    if isinstance(source, list):
        return ''.join(source)
    return source

## Testing

Let's test our utilities on the current project:

In [ ]:
# Test getting notebook files
notebooks = get_notebook_files()
print(f"Found {len(notebooks)} notebooks:")
for nb in notebooks[:5]:  # Show first 5
    print(f"  - {nb.name}")

Found 9 notebooks:
  - 00_core.ipynb
  - 01_parsers.ipynb
  - 02_tree.ipynb
  - 03_api_docs.ipynb
  - 04_dependencies.ipynb


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()